# IPAI Homework 1. Taktil Minimax AI

#### Work done by Pavel Tishkin, p.tishkin@innpolis.university

## Task description

The task was to implement a minimax bot for the game [Taktikl](http://cyclowiki.org/wiki/%D0%A2%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%BB%D1%8C). The solution provided below is mainly based on the [TicTacToe bot template](https://github.com/hsu-ai-course/hsu.ai/blob/master/code/02.%20Ti%D1%81-Ta%D1%81-Toe.ipynb), given on the lecture

## Game Analysis

One of the main challenges, connected with the game is that loops of field states are possible with this set of rules. In this work, we can assume that rational player, in circumstances of the possibly finite game, would never choose to go to the loop as it yields no result. Therefore, we can remove connection, when the child point to the already visited node down the path.

To detect the loop, we would need to uniquely define a state as a:

1. Placement of the pieces on the board.
2. Whose turn it is.

Now, let us prove that **2** can be derived from the point **1**.

Let us take the L1 norm for all pieces with their respective starting points. Each of the players moves adds or subtracts 1 from L1 norms of one piece. Therefore, one move of a player changes the parity of the total sum. Let us prove that this sum is always even at the start of Player1's turn by induction:

1. Player One starts with an even total sum (0)
2. If at turn *N* of Player1 his sum was  equal to *x* and odd, at *N+1*th turn his sum would become *x+2* which is also an odd number

The same logic could be applied to prove that Player2 starts with the odd sum.

It is also worth mentioning, given the setup, L1 distances between the starting points of the pieces are even numbers. This simplifies the procedure of finding out the parity of the sum of distances. We can calculate the distances between the pieces and a one of respective player's starting point. 

Let us assume there exists such a board with the same placement of the pieces, but different player's turn. The Manhattan distance should be odd in one case and even in the other. Therefore, exists such a position on the field, to which one path takes an even number of steps and the other one takes odd number of steps. Then, let us take an arbitrary piece and move to the field using odd number of steps path and return to the starting point. The resulting number of steps will be odd. But that is not possible with the setup of the game. Since each step increases or decreases the Manhattan distance by one, it takes the same amount of steps to go away from the point as the amount of steps to return to the point. Therefore, such position does not exist and the placement of pieces on the board can uniquely identify whose turn it is.

## Coding part

As a result of the analysis, we can build a tree with field states as indeces and check for loops each time we get to the same field.

To get whose turn it is we can check the parity of the sum of L1 norms.

To check the win we can find an area of triangle consisting of three points. If it equals to zero and points are located next to one another, one of the players wins

However, checking on loops each time, checking for possible moves is computationally expensive. It may take a significant amount of time to calculate the whole tree, which is probably not the best decision. It would be more convenient to move to some limited depth of, lets say, 5.

In [1]:
import copy
import itertools
import numpy as np

def tr_area(x):
    '''
    Getting area of a triangle by 3 points. 
    Useful in checking the winning condition and in updating minimax cost
    x - [3, 2] array of points
    Reference: https://www.vedantu.com/maths/collinear-points
    '''
    return np.linalg.det(np.array([[x[0][0], x[0][1], 1], [x[1][0], x[1][1], 1], [x[2][0], x[2][1], 1]]))

class Field(object):
    # state is a 4x4 char matrix
    # Game pieces are going to be represented by:
    # 'x' - player 1's piece
    # 'o' - player 2's piece
    state = None
    children = []
    minimax_score = None
    x_points = None
    o_points = None
    
    def __init__(self, state):
        self.state = state
        self.x_points = np.argwhere(np.array(state) == 'x')
        self.o_points = np.argwhere(np.array(state) == 'o')
        
    def __str__(self):
        '''
        this is a field representation code
        '''
        
        res = "  | A | B | C | D |\n-------------------\n"
        for i, line in enumerate(self.state):
            res += "{} | {} | {} | {} | {} |\n".format(i+1, *line)
            res += "-------------------\n"
        return res
    
    def __repr__(self):
        '''
        this is a short form to represent a field
        '''
        return "".join(itertools.chain(*self.state))
    
    def get(self, tpl):
        # Done
        '''
        returns a characted in a given position
        '''
        return self.state[tpl[0]][tpl[1]]
    
    def make_a_move(self, tpl):
        '''
        Makes a move and returns a new field
        '''
        who = self.is_move_of()
        state = copy.deepcopy(self.state)
        state[tpl[0]][tpl[1]] = ' '
        state[tpl[2]][tpl[3]] = who
        return Field(state)
    
    def is_move_of(self):
        '''
        returns a piece if move can be done and a None if game is over
        '''
        if self.is_win_of() is not None:
            return None
        x_moves = sum(map(lambda x: np.linalg.norm(x, ord=1), self.x_points))
        o_moves = sum(map(lambda x: np.linalg.norm((x - np.array([0, 1])), ord=1), self.o_points))
        return 'o' if ((x_moves + o_moves) % 2) == 1 else 'x'
    
    def cross(self, x):
        '''
        Checks if det is 0 and points lie next to each other
        '''
        det = round(tr_area(x))
        if (det == 0 and (max(x[:, 0]) - min(x[:, 0])) <= 2 and (max(x[:, 1]) - min(x[:, 1])) <= 2):
            return True
        return False
    
    def is_win(self, points):
        '''
        for given 4 points of one player checks if 3 of them are winning
        '''
        areas = list(map(lambda x: self.cross(np.array(x)), itertools.combinations(points, 3)))
        return True in areas
    
    def is_win_of(self):
        '''
        Checks who is winning
        '''
        if self.is_win(self.x_points):
            return 'x'
        if self.is_win(self.o_points):
            return 'o'
        # if game is not over -> return None
        return None
    
    def get_next_move(self):
        '''
        Gets the best move for the bot
        '''
        max_score = 0
        best_ch = None
        for ch in self.children:
            if (ch.minimax_score >= max_score):
                # in case of vizhu maat v 4 hoda, a ne v 1
                if (ch.is_win_of() == 'x'):
                    return ch
                max_score = ch.minimax_score
                best_ch = ch
        return best_ch

## Converting moves

In [2]:
def is_possible_move(field, side, start, end):
    '''
    Checks if move is possible
    '''
    if (start[0] > 3 or start[0] < 0):
        return False
    if (start[1] > 3 or start[1] < 0):
        return False
    if (end[0] > 3 or end[0] < 0):
        return False
    if (end[1] > 3 or end[1] < 0):
        return False
    if not ((abs(start[0] - end[0]) == 1 and (start[1] - end[1]) == 0) or (abs(start[1] - end[1]) == 1 and (start[0] - end[0]) == 0)):
        return False
    if not (field.state[start[0]][start[1]] == side and field.state[end[0]][end[1]] == ' '):
        return False
    return True

def get_move_tuple(notation, field):
    '''
    Converts notation like 'c1c2' into a tuple (2, 0, 2, 1)
    '''
    notation = notation.lower()
    if len(notation) != 4:
        return None
    if (notation[0] not in 'abcd') or (notation[1] not in '1234') or (notation[2] not in 'abcd') or (notation[3] not in '1234'):
        return None
    start, end = [int(notation[1]) - 1, {'a': 0, 'b': 1, 'c': 2, 'd': 3,}[notation[0]]], [int(notation[3]) - 1, {'a': 0, 'b': 1, 'c': 2, 'd': 3,}[notation[2]]]
    if is_possible_move(field, 'o', start, end):
        return (start[0], start[1], end[0], end[1])
    return None

## Generating children of the field

In [3]:
import copy

def is_loop(states, start, end):
    '''
    Checks for loops
    '''
    queue = [states[start]]
    while queue:
        state = queue.pop()
        for ch in state.children:
            if repr(ch) == end:
                return True
            queue.append(states[repr(ch)])
    return False
    
def generate_children_for_the_field(field, states):
    '''
    Generates children for the field
    '''
    result = []
    movements = [np.array([1, 0]), np.array([-1, 0]), np.array([0, 1]), np.array([0, -1])]
    if field.is_win_of() is None:
        who_moves = field.is_move_of()
        points = field.x_points if who_moves == 'x' else field.o_points
        for p in points:
            for m in movements:
                new_p = p+m
                if is_possible_move(field, who_moves, p, new_p):
                    state = copy.deepcopy(field.state)
                    state[p[0]][p[1]] = ' '
                    state[new_p[0]][new_p[1]] = who_moves
                    child = Field(state)
                    if (repr(child) in states.keys()):
                        if not is_loop(states, repr(child), repr(field)):
                            result.append(child)
                    else:
                        result.append(child)
    field.children = result
    return result

## Updating minimax

In [4]:
def update_minimax(field):
    '''
    Updates minimax
    If the state is not terminal but there it is max depth
    assigns a value that is better the less area of the polygon
    of the points of the player 'x' 10000- area
    Win - 10000
    lose - -10000
    '''
    field.minimax_score = 0
    win = field.is_win_of()
    if win != None:
        if win == 'x':
            field.minimax_score = 10000
        if win == 'o':
            field.minimax_score = -10000
    else:
        side = field.is_move_of()
        scores = []
        for ch in field.children:
            update_minimax(ch)
            scores.append(ch.minimax_score)
        if scores:
            if side == 'x':
                field.minimax_score = max(scores)
            else:
                field.minimax_score = min(scores)
        else:
            p1, p2 = field.x_points[:3], field.x_points[1:]
            field.minimax_score = 10000 - abs(tr_area(p1)) - abs(tr_area(p2))
    return field.minimax_score

## Generating a Tree

In [5]:
# generate a tree
# store it as a map {str -> Field}
def generate_tree(initial, depth=5, debug=False):
    '''
    Generates a tree
    '''
    states = {initial.__repr__(): initial}
    queue = [(initial, 1)]

    while queue:
        curr = queue.pop()
        children = generate_children_for_the_field(curr[0], states)
        for child in children:
            states[child.__repr__()] = child
            if (depth > curr[1]):
                queue.append((child, curr[1] + 1))

    update_minimax(initial)
    if debug:
        print('Total states:', len(states))
        ## BTW, is this ok that some nodes in a tree have the same repr?
        print("Root score: ", initial.minimax_score)
    
    return states

## Game Engine

In [6]:
from IPython.display import clear_output

state0 = [['x', 'o', 'x', 'o'], [' ', ' ', ' ', ' '], [' ', ' ', ' ', ' '], ['o', 'x', 'o', 'x']]

initial = Field(state0)

field = initial
while field.is_win_of() is None:
    # make a bot move
    states = generate_tree(field, depth=5)
    field = field.get_next_move()
    # show it
    print(field)
    # if bot wins
    if field.is_win_of() is not None:
        break
    # ask for a move
    m = input("Your move:")
    tpl = get_move_tuple(m, field)
    while tpl is None:
        print("Provide something like `b3b2` of an empty field")
        m = input("Your move:")
        tpl = get_move_tuple(m, field)
    # first build a representation, then retrieve a field from the tree
    field = states[field.make_a_move(tpl).__repr__()]
        
print(field.is_win_of(), "wins")

  | A | B | C | D |
-------------------
1 | x | o | x | o |
-------------------
2 |   |   |   |   |
-------------------
3 |   |   |   | x |
-------------------
4 | o | x | o |   |
-------------------



Your move: c4c3


  | A | B | C | D |
-------------------
1 | x | o |   | o |
-------------------
2 |   |   | x |   |
-------------------
3 |   |   | o | x |
-------------------
4 | o | x |   |   |
-------------------



Your move: b1b2


  | A | B | C | D |
-------------------
1 |   | x |   | o |
-------------------
2 |   | o | x |   |
-------------------
3 |   |   | o | x |
-------------------
4 | o | x |   |   |
-------------------

x wins
